# Config

The configuration is a local file. I'll need:

* namespacing ([Default], others)
* root, default ~/.data_lab/slips, mkdir on that stuff
* slips_directory: slips
* indexes_directory: indexes (full-text indexes)
* citations_directory: citations
* user: name, email
* templates_dictory: templates
* outlines_directory: outlines

Will want:

create_system_directories
nuke_slips
copy_system

In [1]:
from slip_box.imports import *

## Build Configuration

Make it easy to build a configuration file, read one, update it, validate it, and generally guide the slip box system from configuration.

In [2]:
def listify(o):
    """Create lists from objects"""
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, dict): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

def get_deep(o, keys, default=None):
    if not hasattr(o, 'get'): return default
    keys = listify(keys)
    if len(keys) == 0: return default
    key = keys.pop(0)
    value = o.get(key, '__not_found__')
    if value == '__not_found__': return default
    if len(keys) == 0: return value
    return get_deep(value, keys, default=default)
    
    
def build_config(path=None, config=None, **sections):
    """Setup a config object.
    Will read a file, if provided.
    Will replace sections, if provided."""
    
    # TODO: Add validation, defaults, type management
    if config is None:
        if not path is None:
            config = ConfigObj(str(path))
        else:
            config = ConfigObj()

    for section, d in sections.items():
        if not isinstance(d, dict): continue
        config[section] = d
    return config


In [3]:
# Default config is empty.
config = build_config()
assert len(config) == 0
assert config.filename is None

# get_deep helper method works.
config = build_config(foo=dict(bar=dict(baz=dict(thud='thunk'))))
assert get_deep(config, 'foo bar baz thud'.split()) == 'thunk'
assert get_deep(config, 'foo bar baz leroy'.split(), default=42) == 42

# Reading a file works, though without validation the type is string.
try:
    filename = "/tmp/test_config.ini"
    remove_file(filename)
    config = build_config(path=filename, main=dict(a=1, b=2))
    config.write()
    config = build_config(path=filename)
    assert get_deep(config, 'main a'.split()) == '1'
    assert get_deep(config, 'main b'.split()) == '2'
    
finally:
    remove_file(filename)

# Rebuilding doesn't attempt to merge
rebuilt = build_config(config, main=dict(b=2, c=3))
assert get_deep(rebuilt, 'main a'.split()) is None
assert get_deep(rebuilt, 'main b'.split()) == 2

### Making Sense

The ConfigObj tool is worth it. It's simple enough to use, exposes the data well, and has fewer quirks than ConfigParser.

The build_config function is nice because I can fold in validation and type assertions. Right now, it handles in-memory and from-file configuration. In order to save the file, assuming filename is set, call config.write(). To set the filename, just assign it: config.filename = ....

### TODO

* Write the [validation](http://www.voidspace.org.uk/python/articles/configobj.shtml#validation). 
* Save slips from the [ConfigObj docs](http://www.voidspace.org.uk/python/configobj.html)
* Get defaults from validation, or else come up with something else.
* Merge section values, when merge=True
* Remove unset sections, when flush=True

## Slip Box Configuration

* Implement a default configuration in a default location.
* Make sure the resulting object is easy to use.
* Decide whether to include this in imports.

In [8]:
root = Path('/tmp')
path = root/"e1.ini"

In [44]:
defaults = dict(
    root="~/.data_lab/slip_box",
    slips_direcotry="slips",
    indexes_direcotry="indexes",
    citations_direcotry="citations",
    templates_direcotry="templates",
    outlines_direcotry="outlines",
)

tests = {'root': '/tmp/.data_lab/slip_box'}